<a href="https://colab.research.google.com/github/Yiqing-Wang-05/infosci301_final_project/blob/main/Final_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving GDP.xlsx to GDP.xlsx


In [2]:
from google.colab import files
uploaded = files.upload()

Saving Government expenditure on education as % of GDP (%).xlsx to Government expenditure on education as % of GDP (%).xlsx


In [3]:
from google.colab import files
uploaded = files.upload()

Saving inbound and outbound of international students.xlsx to inbound and outbound of international students.xlsx


In [4]:
from google.colab import files
uploaded = files.upload()

Saving outbound data.xlsx to outbound data.xlsx


In [5]:
from google.colab import files
uploaded = files.upload()

Saving Urban population (% of total population).xlsx to Urban population (% of total population).xlsx


In [11]:
from google.colab import files
uploaded = files.upload()

Saving Total inbound internationally mobile students, both sexes (number).xlsx to Total inbound internationally mobile students, both sexes (number).xlsx


In [28]:
# 📘 Animated Map: Inbound Student Migration & Contextual Indicators

import pandas as pd
import plotly.express as px

# --- Function to clean World Bank-style data ---
def fix_and_clean_worldbank_data(path, value_name):
    df = pd.read_excel(path)
    df.columns = df.iloc[3]
    df = df[4:]
    df = df.rename(columns={'Country Name': 'Country', 'Country Code': 'Country Code'})
    df_melted = df.melt(id_vars=['Country', 'Country Code'], var_name='Year', value_name=value_name)
    df_melted['Year'] = pd.to_numeric(df_melted['Year'], errors='coerce')
    df_melted.dropna(subset=['Year', value_name], inplace=True)
    return df_melted

# --- Load cleaned datasets ---
gdp_clean = fix_and_clean_worldbank_data('/content/GDP.xlsx', 'GDP (US$)')
edu_exp_clean = fix_and_clean_worldbank_data('/content/Government expenditure on education as % of GDP (%).xlsx', 'Education Expenditure (% of GDP)')
urban_clean = fix_and_clean_worldbank_data('/content/Urban population (% of total population).xlsx', 'Urban Population (%)')

# --- Clean inbound dataset (manually formatted, no headers) ---
inbound_alt = pd.read_excel("/content/Total inbound internationally mobile students, both sexes (number).xlsx", header=None)
inbound_alt = inbound_alt.iloc[:, :4]
inbound_alt.columns = ['Source ID', 'Country Code', 'Year', 'Inbound Students']
inbound_alt['Inbound Students'] = pd.to_numeric(inbound_alt['Inbound Students'], errors='coerce')
inbound_alt['Year'] = pd.to_numeric(inbound_alt['Year'], errors='coerce')
inbound_alt.dropna(subset=['Year', 'Inbound Students'], inplace=True)
inbound_alt['Year'] = inbound_alt['Year'].astype(int)

# --- Add country names via GDP reference ---
ref = gdp_clean[['Country', 'Country Code']].drop_duplicates()
inbound_alt = pd.merge(inbound_alt, ref, on='Country Code', how='left')

# --- Clean outbound data ---
outbound = pd.read_excel('/content/outbound data.xlsx')
outbound_clean = outbound[
    (outbound['Sex'] == 'All genders') &
    (outbound['Age group'] == 'Not applicable') &
    (outbound['Units of measurement'] == 'Number')
][['Reference Area', 'Time Period', 'Observation Value']].copy()
outbound_clean.columns = ['Country', 'Year', 'Outbound Students']
outbound_clean['Year'] = pd.to_numeric(outbound_clean['Year'], errors='coerce')

# --- Merge all datasets ---
merged = inbound_alt[['Country', 'Year', 'Inbound Students']]\
    .merge(outbound_clean, on=['Country', 'Year'], how='outer')\
    .merge(gdp_clean, on=['Country', 'Year'], how='outer')\
    .merge(edu_exp_clean, on=['Country', 'Year'], how='outer')\
    .merge(urban_clean, on=['Country', 'Year'], how='outer')

# --- Add coordinates manually (faster and reliable) ---
country_coords = pd.DataFrame({
    "Country": ["China", "India", "Japan", "Korea, Rep.", "Malaysia", "Vietnam", "Pakistan",
                "Bangladesh", "Indonesia", "Thailand", "United States", "United Kingdom",
                "Germany", "France", "Canada", "Australia"],
    "Latitude": [35.8617, 20.5937, 36.2048, 35.9078, 4.2105, 14.0583, 30.3753,
                 23.6850, -0.7893, 15.8700, 37.0902, 55.3781,
                 51.1657, 46.6034, 56.1304, -25.2744],
    "Longitude": [104.1954, 78.9629, 138.2529, 127.7669, 101.9758, 108.2772, 69.3451,
                  90.3563, 113.9213, 100.9925, -95.7129, -3.4360,
                  10.4515, 1.8883, -106.3468, 133.7751]
})
merged = merged.merge(country_coords, on='Country', how='left')

# --- Filter data for animation (must have inbound, lat/lon) ---
df_latest = merged[
    (merged['Inbound Students'] > 0) &
    (~merged['Latitude'].isna()) &
    (~merged['Longitude'].isna())
].copy()
df_latest['Year'] = df_latest['Year'].astype(int)
df_latest = df_latest[df_latest['Year'].between(2000, 2022)]
df_latest['Year'] = df_latest['Year'].astype(str)

# --- Animated map ---
fig = px.scatter_geo(
    df_latest,
    lat='Latitude', lon='Longitude',
    size='Inbound Students',
    color='Education Expenditure (% of GDP)',
    hover_name='Country',
    hover_data={
        'Inbound Students': ":,.0f",
        'Outbound Students': ":,.0f",
        'GDP (US$)': ":,.0f",
        'Urban Population (%)': ':.1f'
    },
    animation_frame='Year',
    projection='natural earth',
    size_max=40,
    template='plotly_white',
    title="🌍 Inbound Student Migration Over Time<br><sub>Size = Inbound Volume, Color = Education Investment</sub>"
)

fig.update_geos(
    showcountries=True, countrycolor="lightgray",
    showland=True, landcolor="whitesmoke",
    showocean=True, oceancolor="lightblue"
)

fig.update_layout(
    margin=dict(l=0, r=0, t=60, b=0),
    updatemenus=[{
        'buttons': [
            {'args': [None, {"frame": {"duration": 1500, "redraw": True}, "fromcurrent": True}],
             'label': "Play", 'method': "animate"},
            {'args': [[None], {"frame": {"duration": 0, "redraw": False},
                               "mode": "immediate", "transition": {"duration": 0}}],
             'label': "Pause", 'method': "animate"}
        ],
        'direction': "left",
        'pad': {"r": 10, "t": 70},
        'showactive': True,
        'type': "buttons",
        'x': 0.1, 'xanchor': "right",
        'y': 0, 'yanchor': "top"
    }]
)

fig.show()